In [1]:
import os
import pandas as pd
import cv2
import glob
import time
import math
import numpy as np
from PIL import Image
import keras
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input,Flatten,Dropout
from keras.models import Model, Sequential
from keras.applications import ResNet50
from keras.applications import ResNet152
from keras.layers import concatenate
from keras.utils import plot_model
#Inception Model
from keras.applications import InceptionV3
from keras.utils import plot_model

In [2]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 1.75,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 1.75,
                         (img_height + crop_height) // 2))


def crop_max_square(pil_img):
    return crop_center(pil_img, min(pil_img.size), min(pil_img.size))
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

def append_fileinfo(fn,path):
    return path + fn+".jpg"

def append_superpixelinfo(fn,path):
    return path + fn + ".png"    

dir(keras.applications)

['ConvNeXtBase',
 'ConvNeXtLarge',
 'ConvNeXtSmall',
 'ConvNeXtTiny',
 'ConvNeXtXLarge',
 'DenseNet121',
 'DenseNet169',
 'DenseNet201',
 'EfficientNetB0',
 'EfficientNetB1',
 'EfficientNetB2',
 'EfficientNetB3',
 'EfficientNetB4',
 'EfficientNetB5',
 'EfficientNetB6',
 'EfficientNetB7',
 'EfficientNetV2B0',
 'EfficientNetV2B1',
 'EfficientNetV2B2',
 'EfficientNetV2B3',
 'EfficientNetV2L',
 'EfficientNetV2M',
 'EfficientNetV2S',
 'InceptionResNetV2',
 'InceptionV3',
 'MobileNet',
 'MobileNetV2',
 'MobileNetV3Large',
 'MobileNetV3Small',
 'NASNetLarge',
 'NASNetMobile',
 'ResNet101',
 'ResNet101V2',
 'ResNet152',
 'ResNet152V2',
 'ResNet50',
 'ResNet50V2',
 'ResNetRS101',
 'ResNetRS152',
 'ResNetRS200',
 'ResNetRS270',
 'ResNetRS350',
 'ResNetRS420',
 'ResNetRS50',
 'VGG16',
 'VGG19',
 'Xception',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'convnext',
 'densenet',
 'efficientnet',
 'efficientnet_v2',
 '

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/ResizedImages-20230403T182719Z-001/ResizedImages/"

Mounted at /content/drive
ISIC_0000000.jpg  ISIC_0009974.jpg  ISIC_0013023.jpg
ISIC_0000001.jpg  ISIC_0009975.jpg  ISIC_0013024.jpg
ISIC_0000002.jpg  ISIC_0009976.jpg  ISIC_0013025.jpg
ISIC_0000003.jpg  ISIC_0009977.jpg  ISIC_0013026.jpg
ISIC_0000004.jpg  ISIC_0009979.jpg  ISIC_0013027.jpg
ISIC_0000006.jpg  ISIC_0009981.jpg  ISIC_0013031.jpg
ISIC_0000007.jpg  ISIC_0009982.jpg  ISIC_0013032.jpg
ISIC_0000008.jpg  ISIC_0009987.jpg  ISIC_0013033.jpg
ISIC_0000009.jpg  ISIC_0009991.jpg  ISIC_0013034.jpg
ISIC_0000010.jpg  ISIC_0009993.jpg  ISIC_0013037.jpg
ISIC_0000011.jpg  ISIC_0009994.jpg  ISIC_0013039.jpg
ISIC_0000012.jpg  ISIC_0009998.jpg  ISIC_0013040.jpg
ISIC_0000013.jpg  ISIC_0010000.jpg  ISIC_0013044.jpg
ISIC_0000014.jpg  ISIC_0010002.jpg  ISIC_0013047.jpg
ISIC_0000015.jpg  ISIC_0010003.jpg  ISIC_0013048.jpg
ISIC_0000016.jpg  ISIC_0010005.jpg  ISIC_0013052.jpg
ISIC_0000017.jpg  ISIC_0010006.jpg  ISIC_0013053.jpg
ISIC_0000018.jpg  ISIC_0010011.jpg  ISIC_0013054.jpg
ISIC_0000019.jpg  IS

In [4]:
#Create Partial Data Set
#datapath="./ISIC-2017_Training_Data/"
datapath2 = "./generatedimages"
#datapath = "./imageTest/"
#datapath = "/content/drive/My Drive/Colab Notebooks/imageTest/"
datapath = "/content/drive/My Drive/ResizedImages-20230403T182719Z-001/ResizedImages/"


#filename_read = os.path.join(datapath + "ISIC-2017_Training_Part3_ModGroundTruth.csv")    
filename_read = datapath + "ISIC-2017_Training_Part3_ModGroundTruth.csv"     
dfBase = pd.read_csv(filename_read)
print("Done reading image filenames")
#Seperate Categories into dataframes
dfNevus = dfBase[dfBase.Nevus == 'Nevus']
dfMelanoma = dfBase[dfBase.melanoma == 'melanoma']
dfSeb = dfBase[dfBase.seborrheic_keratosis == 'seborrheic_keratosis']

#Take smaller set of data and increase representation of Melanoma and Seb records to help with imbalance
dfNevus = dfNevus[0:500]
dfMelanoma = dfMelanoma[0:250]
dfSeb = dfSeb[0:150]
df = pd.concat([dfNevus, dfMelanoma, dfSeb])

df=shuffle(df)

#df=dfBase
df['filename'] = df['image_id'].apply(append_fileinfo,path=datapath)
#df['filename'] = df['image_id'].apply(append_superpixelinfo,path=datapath)
df = df.drop(['image_id','melanoma', 'seborrheic_keratosis','Nevus'],axis=1)

df

Done reading image filenames


,Category,filename
461,melanoma,/content/drive/My Drive/ResizedImages-20230403...
1073,seborrheic_keratosis,/content/drive/My Drive/ResizedImages-20230403...
177,Nevus,/content/drive/My Drive/ResizedImages-20230403...
473,melanoma,/content/drive/My Drive/ResizedImages-20230403...
157,melanoma,/content/drive/My Drive/ResizedImages-20230403...
...,...,...
1171,seborrheic_keratosis,/content/drive/My Drive/ResizedImages-20230403...
781,melanoma,/content/drive/My Drive/ResizedImages-20230403...
21,melanoma,/content/drive/My Drive/ResizedImages-20230403...
1641,seborrheic_keratosis,/content/drive/My Drive/ResizedImages-20230403...


In [5]:
classlist = ["melanoma", "seborrheic_keratosis","Nevus"]


datagen = ImageDataGenerator(rescale=1./255,rotation_range=180,
                             data_format='channels_last', validation_split=0.25)


train_inc = datagen.flow_from_dataframe(df, x_col='filename', y_col='Category',  classes=classlist, 
                                    class_mode='categorical',  batch_size=32, shuffle=False, 
                                    target_size=(299, 299), save_format='png',
                                   subset="training")

validate_inc =datagen.flow_from_dataframe(df, x_col='filename', y_col='Category', classes=classlist, 
                                    class_mode='categorical',  batch_size=32, shuffle=False, 
                                    target_size=(299, 299), save_format='png',
                                   subset="validation")



#inp_gen= generator_two_img(we will pass the data here)

Found 675 validated image filenames belonging to 3 classes.
Found 225 validated image filenames belonging to 3 classes.


In [6]:
print("num training recs = {}" .format(train_inc.n))
print("size training batch = {}" .format(train_inc.batch_size))
TRAIN_RECS = train_inc.n
BATCH_SIZE = train_inc.batch_size
VALIDATE_RECS = validate_inc.n
TRAIN_STEPS = math.floor(TRAIN_RECS/BATCH_SIZE)
VALIDATE_STEPS = math.floor(VALIDATE_RECS/BATCH_SIZE)
print("Train steps = {}, Validate steps = {}" . format(TRAIN_STEPS, VALIDATE_STEPS))

num training recs = 675
size training batch = 32
Train steps = 21, Validate steps = 7


In [7]:
#Define Models

#Resnet Model Contribution
input_tensor = Input(shape=(299,299,3))
input_tensor_model = Input(shape=(BATCH_SIZE,299,299,3))
reslayer = ResNet152(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(299,299,3), pooling=None, classes=3)
for layer in reslayer.layers:
  layer.trainable = False 
rl1 = reslayer.output
rflat = Flatten()(rl1)
rdense = Dense(1024,activation='relu')(rflat)
rdense2 = Dense(512, activation='relu')(rdense)
rdropout=Dropout(.1)(rdense2)
rdense3 = Dense(256, activation='relu')(rdropout)
#rprediction = Dense(3, activation='softmax')(rdropout)
#rfinal_model = Model(inputs=reslayer.input, outputs=rprediction)
#final_model.summary()




#input_tensor = Input(shape=(299,299,3))
inclayer = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(299,299,3), pooling=None, classes=3)
for layer in inclayer.layers:
  layer.trainable = False 
inc_l1 = inclayer.output
inc_dense = Dense(24,activation='relu')(inc_l1)
inc_dropout=Dropout(.25)(inc_dense)
inc_dense2 = Dense(12, activation='relu')(inc_dropout)
inc_dropout2 = Dropout(.25)(inc_dense2)
inc_flat = Flatten()(inc_dropout2)
inc_dropout3 = Dropout(.25)(inc_flat)
inc_dense3 = Dense(256, activation='relu')(inc_dropout3)
#inc_prediction = Dense(3, activation='softmax')(inc_dropout3)
#inc_final_model = Model(inputs=inclayer.input, outputs=inc_prediction)
#inc_final_model.summary()

# merge feature extractors = combine output from two CNN models
merge = concatenate([inc_dense3, rdense3])

# interpretation layers
hidden1 = Dense(512, activation='relu')(merge)
hidden1 = Dense(256, activation='relu')(hidden1)
dropout = Dropout(.25)(hidden1)
hidden3 = Dense(128, activation='relu')(dropout)

# prediction output
prediction = Dense(3, activation='softmax')(hidden3)

final_model = Model(inputs=input_tensor, outputs=prediction)
final_model.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 305, 305, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                             

In [8]:
# plot graph
plot_model(final_model, to_file='./multiple_inputs.png')

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.650384 to fit



In [9]:
#Run Two-way Model - Try slower learning rate to get rid of prediction errors
checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/bestTwoChannelLess_weights.hdf5", verbose=0, save_best_only=True) # save best model
opt = keras.optimizers.Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=3, verbose=0, mode='auto',restore_best_weights=True) 
final_model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=['accuracy'])


start_time = time.time()
print("Start time: {}",format(hms_string(start_time)))
final_model.fit(generator=train_inc,
                    steps_per_epoch=TRAIN_STEPS,
                    validation_data=validate_inc,
                    validation_steps=VALIDATE_STEPS,
                    epochs=100,
                    workers=3,
                    use_multiprocessing=False,
                    max_queue_size=4,
                    callbacks=[monitor,checkpointer])

elapsed_time = time.time() - start_time
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print("Elapsed time: {}".format(elapsed_time))    

Start time: {} 466935:29:31.96


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-9-de316bdb0d0e>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final_model.fit_generator(generator=train_inc,


Epoch 1/100
21/21 [==============================] - 1299s 61s/step - loss: 1.6160 - accuracy: 0.4386 - val_loss: 1.0918 - val_accuracy: 0.5625
Epoch 2/100
21/21 [==============================] - 1277s 61s/step - loss: 1.4038 - accuracy: 0.4339 - val_loss: 1.0074 - val_accuracy: 0.5536
Epoch 3/100
21/21 [==============================] - 1223s 58s/step - loss: 1.2407 - accuracy: 0.4697 - val_loss: 1.0408 - val_accuracy: 0.5625
Epoch 4/100
21/21 [==============================] - 1214s 58s/step - loss: 1.1326 - accuracy: 0.4650 - val_loss: 1.0098 - val_accuracy: 0.5536
Epoch 5/100
21/21 [==============================] - 1205s 58s/step - loss: 1.0682 - accuracy: 0.4868 - val_loss: 1.1166 - val_accuracy: 0.2634
Epoch 6/100
21/21 [==============================] - 1189s 57s/step - loss: 1.1044 - accuracy: 0.4681 - val_loss: 1.0271 - val_accuracy: 0.4598
Epoch 7/100
21/21 [==============================] - 1187s 57s/step - loss: 1.0535 - accuracy: 0.5210 - val_loss: 1.0506 - val_accuracy:

In [10]:
#Perform Prediction
#monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=3, verbose=0, mode='auto',restore_best_weights=True) 
#final_model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=['accuracy'])
test_generator =validate_inc
#final_model.load_weights("/content/drive/My Drive/Colab Notebooks/bestTwoChannelLess_weights.hdf5")
pred = final_model.predict_generator(test_generator, steps=VALIDATE_STEPS)
rawpred = np.argmax(pred,axis=1)
print(pred)
print(rawpred)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in rawpred]
print(predictions)

<ipython-input-10-f527852860e7>:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = final_model.predict_generator(test_generator, steps=VALIDATE_STEPS)


[[0.3840226  0.3323766  0.28360087]
 [0.40484384 0.31206393 0.2830923 ]
 [0.4126773  0.30974817 0.27757463]
 [0.40084508 0.33461636 0.26453853]
 [0.28732142 0.10526597 0.6074125 ]
 [0.27591002 0.22581822 0.49827155]
 [0.39491084 0.270582   0.33450717]
 [0.40644127 0.41560265 0.17795607]
 [0.32763258 0.13635358 0.5360138 ]
 [0.38710508 0.40290287 0.20999219]
 [0.41998544 0.25883794 0.32117644]
 [0.38965377 0.23590884 0.37443742]
 [0.2870119  0.2002772  0.5127109 ]
 [0.35608247 0.3244241  0.3194934 ]
 [0.38431093 0.33112594 0.2845632 ]
 [0.35419258 0.20721485 0.43859246]
 [0.33794177 0.11657615 0.54548216]
 [0.44848725 0.28771842 0.26379424]
 [0.32455662 0.10252009 0.57292324]
 [0.37573332 0.3290221  0.29524454]
 [0.32409042 0.15977469 0.51613486]
 [0.3143125  0.14002207 0.5456655 ]
 [0.37962672 0.39886674 0.22150654]
 [0.28635302 0.24348374 0.47016326]
 [0.34387237 0.18849938 0.46762827]
 [0.35931805 0.06410407 0.5765778 ]
 [0.50337017 0.18422821 0.31240168]
 [0.4066781  0.27280837 0.32

In [ ]:
#Evaluate Model
eval = final_model.evaluate_generator(test_generator, steps=VALIDATE_STEPS)
print(final_model.metrics_names)
eval

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
l= test_generator.class_indices.keys()
print(l)
y_tru = test_generator.classes[0:224]
cm = confusion_matrix(y_tru, rawpred)
print(cm)
print(classification_report(y_tru, rawpred))

dict_keys(['melanoma', 'seborrheic_keratosis', 'Nevus'])
[[39  3 17]
 [27  7  5]
 [48  1 77]]
              precision    recall  f1-score   support

           0       0.34      0.66      0.45        59
           1       0.64      0.18      0.28        39
           2       0.78      0.61      0.68       126

    accuracy                           0.55       224
   macro avg       0.59      0.48      0.47       224
weighted avg       0.64      0.55      0.55       224



In [14]:
#AUC Metrics
def convert_tobinary(x):
    if (x > 0).bool():
        return 1
    else:
        return 0

#Convert 3 classification to binary for ROC_AUC
dftrue = pd.DataFrame(y_tru, dtype=int);
dfpred = pd.DataFrame(rawpred, dtype=int);
dftrue = dftrue.apply(convert_tobinary,axis=1)
dfpred = dfpred.apply(convert_tobinary,axis=1)

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true=dftrue,  y_score=dfpred, average='samples')

0.6032357473035439